## Read files like Be8_SIII_new.txt

**Naming convention: YYMMDD_Be8**

To use this notebook to generate input for vqe:
1. Provides source file name labeled(#filename)
2. Parameters are to be output in the format of a list 
```
the list = [num_orbitals,num_spin_orbitals,num_particles,num_spatial_orbitals]
```

In [1]:
from utils import *
## Computation config ## 
pcname = "Hpc" #or "Hlp" or Ypc"
quan_algo = "VQE"    ## (string)(VQE or adaptVQE)
## Source File input ##
input_dir = "230412_Be8"

### The following doesnt require setting up, unless file naming is different
obs_onebody_csv = input_dir+"/"+input_dir+"-1B_H_input.csv"
obs_twobody_csv = input_dir+"/"+input_dir+"-2B_H_input.csv"
parameter_py = input_dir+"/"+"parameter.txt"

def pathfilename_gen(pcname_:"string", input_dir_:"string")->"string , string , dict":
    ## Setting up the path (now is directory where compute.py is ran)
    current_path = os.getcwd()                     # Current path(which is the Compute dir)
    try:
        abs_main                                   # Check if the abs_main was defined before
    except:
        abs_main = os.path.dirname(current_path)   # Main directory containing the compute dir
    os.chdir(abs_main)                             # Change dir to main dir(so called absolute main directory)

    abspath_data_dir = os.path.join(abs_main,"Data")
    abspath_result_dir = os.path.join(abs_main,"Result")
    
    #### Extract index for next dir and filename
    rel_path_result = os.path.relpath("Result",abs_main)
    rel_PATH_result = (Path(rel_path_result))
    subresult_dir_list = [str(x) for x in rel_PATH_result.iterdir() if x.is_dir()]
    if len(subresult_dir_list) == 0:
        new_index = 1
    else:
        latest_result_dir = (max(subresult_dir_list,key=extract_number))
        new_index = extract_number(latest_result_dir)[0]+1
    
    #### Create a new directory to keep new results
    ##### Extract neucleus name
    input_dir_dum = re.split('-|_|\\.', input_dir_)
    nucleus_name = input_dir_dum[1]
    output_id = pcname_ + "_" + "{:03d}".format(new_index) + "_"
    subresult_dir_name = os.path.join(rel_path_result,output_id+nucleus_name)
    os.mkdir(subresult_dir_name)
    subresult_dir = os.path.join(subresult_dir_name,output_id+nucleus_name)
    ### Using a dictionary to store the file names to be used
    pathfilename = {}
    #### Input filenames or path
    pathfilename["source_text"] = os.path.join(abspath_data_dir, input_dir_)
    #### Output # only ready to be input as fermionic op is considered not source
    pathfilename["output_1B-source_csv"] =  subresult_dir + "-1B-source.csv"
    pathfilename["output_2B-source_csv"] =  subresult_dir + "-2B-source.csv"
    pathfilename["output_1B-H_input_csv"] = subresult_dir + "-1B-H_input.csv"
    pathfilename["output_2B-H_input_csv"] = subresult_dir + "-2B-H_input.csv"
    pathfilename["abstract_result"] = subresult_dir + "-vqe_abst.txt"
    pathfilename["full_result"] = subresult_dir + "-vqe_full.txt"
    return abs_main, nucleus_name, pathfilename

# abs_main, nucleus, pathfilename = pathfilename_gen(pcname,input_dir)
# pathfilename

In [2]:
parameter_py = "230412_Be8\parameter.txt"
parameter_txt = []


In [3]:
from utils import *

In [4]:
text_two = []
with open ("source/Be8_SIII_new/Vpair_Vdelta_800.txt", 'rt') as myfile:    #filename
    for line in myfile:
        line = line.rstrip('\n')
        text_two.append(line)

# Parameters

In [5]:
num_orbitals = (6,6)
num_particles = (4,4)
num_spin_orbitals = 12
num_spatial_orbitals = int(num_spin_orbitals/2)

In [6]:
import pandas as pd
energy = {"index": [0,1,2,3,4,5,6,7,8,9,10,11], 
          "epsilon":[-26.131,-26.131,-11.415,-11.415,-11.055,-11.055,-24.174,-24.174,-9.613,-9.613,-9.230,-9.230]}

obs_onebody_df_source = pd.DataFrame(energy)

# Twobody Matrix Terms source

In [7]:
import re
pattern = re.compile("isospin")
if pattern.search(text_two[0]) is not None:
    text_two.pop(0)
pairs = [[1,2],[3,4],[5,6]]
obs_twobody_df_source = pd.DataFrame(columns =["q_i1", "q_i2", "q_f1", "q_f2", "V_ffii"])
for counter, line in enumerate(text_two): 
    splitted = line.split()
    if int(splitted[0]) == 1:
        for x in pairs:
            if int(splitted[1]) in x:
                init = x
            if int(splitted[2]) in x:
                fina = x
        init = (init[0]-1,init[1]-1) # minus 1 because the index in python starts with 0
        fina = (fina[0]-1,fina[1]-1)
        V_pair = float(splitted[3])
    if int(splitted[0]) ==2:
        for x in pairs:
            if int(splitted[1]) in x:
                init = x
            if int(splitted[2]) in x:
                fina = x
        init = (init[0]+5,init[1]+5) # plus 5 because the index in python starts with 0; and proton'index starts after neutron's
        fina = (fina[0]+5,fina[1]+5)
        V_pair = float(splitted[3])
    V_pair = round(V_pair,6)
    obs_twobody_df_source.loc[counter] = [init[0],init[1],fina[0],fina[1],V_pair]
    del init, fina
obs_twobody_df_vqe = obs_twobody_df_source.copy()

# Filter out unwanted interactions from obs_twobody_df
# Create obs_twobody_H_input_df_output (input for VQE)

In [8]:
# ### To csv
# obs_twobody_df_vqe.to_csv(pathfilename['output_2B-H_input_csv'])
# obs_twobody_df_vqe

# Create obs_onebody_df_vqe (input for VQE)

In [12]:
obs_onebody_df_source

,index,epsilon
0,0,-26.131
1,1,-26.131
2,2,-11.415
3,3,-11.415
4,4,-11.055
5,5,-11.055
6,6,-24.174
7,7,-24.174
8,8,-9.613
9,9,-9.613


In [25]:
fermin = (-11.415 + (-11.055))/2
fermip = (-9.613 + (-9.230))/2

### Below consider energy differences between levels
obs_onebody_df_vqe = pd.DataFrame(columns =['q_i','q_f','epsilon'])

### init_fina_list
if_list = sin_bod_if_list_gen(num_orbitals)
epsilon_list = obs_onebody_df_source.loc[:,"epsilon"]
import math
for counter, i in enumerate(epsilon_list):
    if counter < num_orbitals[0]:
        epsilon = math.sqrt((i - (fermin))**2)
    if counter >= num_orbitals[0]:
        epsilon = math.sqrt((i - (fermip))**2)
    obs_onebody_df_vqe.loc[counter] = [counter,counter, round(epsilon,6)]

obs_onebody_df_vqe.to_csv("203_Be8/203_Be8-1B_H_input.csv")
obs_onebody_df_vqe

,q_i,q_f,epsilon
0,0.0,0.0,14.8960
1,1.0,1.0,14.8960
2,2.0,2.0,0.1800
3,3.0,3.0,0.1800
4,4.0,4.0,0.1800
5,5.0,5.0,0.1800
6,6.0,6.0,14.7525
7,7.0,7.0,14.7525
8,8.0,8.0,0.1915
9,9.0,9.0,0.1915


In [10]:
# %store promotion_list
# %store num_orbitals
# %store num_particles
# %store num_spin_orbitals
# %store obs_onebody_df_vqe
# %store obs_twobody_df
# %store abs_main
# %store pathfilename
# %store nucleus_name

In [11]:
obs_twobody_df = obs_twobody_df_vqe
from qiskit_nature.second_q.operators import FermionicOp
import pandas as pd
tmp_ham = {}

### Two body Terms: Pairing interaction
if include_twobody == True:
    for index, row in obs_twobody_df.iterrows():
        init_1 = int(row['q_i1']); init_2 = int(row['q_i2']);
        fina_1 = int(row['q_f1']); fina_2 = int(row['q_f2']);
        the_twostring = "+_" +str(fina_1) + " " + "+_" +str(fina_2) + " " + "-_" +str(init_1) + " " + "-_" +str(init_2)
        tmp_ham[the_twostring] = two_factor*row['V_ffii']
len(tmp_ham)

NameError: name 'include_twobody' is not defined

In [ ]:
Hamiltonian = FermionicOp(tmp_ham, 
                          num_spin_orbitals=num_spin_orbitals, 
                          copy=False)


In [ ]:
Hamiltonian